In [2]:
%matplotlib qt5

In [3]:
import nbimporter

In [4]:
from collections import defaultdict, Counter
from itertools import combinations

import numpy as np
from numpy import percentile
import pandas as pd
import pickle
import randomcolor
import pathlib
import shutil
from scipy.stats import percentileofscore
from scipy.spatial.distance import cdist
import random
from random import sample

import itertools
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.decomposition import PCA
from tqdm import tqdm_notebook

from visualization_functions import visualize3DData_hull, visualize2DPCA_hull
from helper_functions import distConsistancy, calcDist, calculateCentroidsMedoids, queryOriginalSpace, clusterMap, pairwiseCentralityDistances

Importing Jupyter notebook from visualization_functions.ipynb
Importing Jupyter notebook from helper_functions.ipynb


# load cases and the metadata

In [5]:
import pickle
with open('gpibdfaceEmbeddingsDict.dict', 'rb') as fh:
    cases = pickle.load(fh)

# add in new cases
    
with open('newCasesEmbeddings', 'rb') as fh:
    
    newCases = pickle.load(fh)

newCases['PIGN_241510'] = newCases.pop('PIGN_65731')

newEmbds = dict(cases)
newEmbds.update(newCases)

_ = newEmbds.pop('GPAA1_willis') # there was something wrong with this case; misdiagnosed or something

case2disease = {c[c.find('_')+1:]:c[:c.find('_')] for c in newEmbds}

case2ethnicity = {'241481': 'AS',
 '241482': 'AS',
 '241483': 'AS',
 '241484': 'AS',
 '241485': 'EU',
 '241486': 'AS',
 '241487': 'EU',
 '241488': 'EU',
 '241489': 'AS',
 '241490': 'EU',
 '241491': 'EU',
 '241492': 'EU',
 '241493': 'EU',
 '241494': 'EU',
 '241495': 'EU',
 '241496': 'EU',
 '241497': 'EU',
 '241498': 'EU',
 '241499': 'EU',
 '241500': 'AS',
 '241501': 'EU',
 '241502': 'EU',
 '241503': 'EU',
 '241504': 'AS',
 '241505': 'AS',
 '241506': 'MEX',
 '241507': 'ARAB',
 '241508': 'AS',
 '241509': 'EU',
 '241510': 'EU',
 '241511': 'EU',
 '241512': 'ARAB',
 '241513': 'EU',
 '241514': 'EU',
 '241515': 'EU',
 '241516': 'TURK',
 '241517': 'TURK',
 '241518': 'TURK',
 '241519': 'EU+AFR',
 '241520': 'EU+AFR',
 '241521': 'NAF',
 '241523': 'NAF',
 '241524': 'EU',
 '241525': 'EU',
 '241526': 'EU',
 '241527': 'EU',
 '241528': 'EU',
 '241529': 'IND',
 '242715': 'EU',
 '242716': 'EU',
 '242717': 'TURK',
 '242718': 'EU',
 '242719': 'TURK',
 '242720': 'MEX',
 '242721': 'MEX',
 '242722': 'NAF',
 '242723': 'NAF',
 '242724': 'FIN',
 '242725': 'EU',
 '242726': 'EU',
 '242727': 'EU',
 '242728': 'EU',
 '242729': 'EU',
 '242730': 'EU',
 '242731': 'EU',
 '242732': 'EU',
 '242733': 'EU',
 '242734': 'EU',
 '242735': 'EU',
 '242736': 'EU',
 '242737': 'EU',
 '242738': 'EU',
 '242739': 'EU',
 '242740': 'EU',
 '242743': 'EU',
 '242744': 'EU',
 '242745': 'EU',
 '242747': 'EU',
 '242748': 'EU',
 '242749': 'EU',
 '242750': 'EU',
 '242751': 'EU',
 '242752': 'EU',
 '242753': 'EU',
 '242754': 'EU',
 '242756': 'NAF',
 '242757': 'EU',
 '242758': 'EU',
 '242759': 'EU',
 '242760': 'EU',
 '242761': 'EU',
 '242762': 'EU',
 '242763': 'IND',
 '242764': 'EU',
 '242765': 'EU',
 '242766': 'NAF',
 '242767': 'NAF',
 '242768': 'NAF',
 '242769': 'NAF',
 '242770': 'NAF',
 '242771': 'NAF',
 '242772': 'NAF',
 '242773': 'NAF',
 '242774': 'NAF',
 '242775': 'NAF',
 '242776': 'NAF',
 '242777': 'NAF',
 '242778': 'EU',
 '242779': 'MID_EA',
 '242780': 'MID_EA',
 '244875': 'EU',
 '244884': 'AS',
 '250288': 'ARAB',
 '250293': 'EU',
 '250300': 'AS'}

case2ethnicity['blondheim'] = 'EU'
case2ethnicity['leszek'] = 'EU'
case2ethnicity['linthicum'] = 'EU'

# for normalizing the naming for all cases; not important

for c,v in list(cases.items()):
    
    tmpc = c[c.find('_')+1:]
    cases[tmpc] = v
    del cases[c]
    
for c,v in list(newEmbds.items()):
    
    tmpc = c[c.find('_')+1:]
    newEmbds[tmpc] = v
    del newEmbds[c]

# (optional) load normal faces

In [8]:
with open('normalFaces', 'rb') as fh:
    
    normal = pickle.load(fh)

wNormal = dict(newEmbds)
wNormal.update(normal)

# load references

In [9]:
with open('referenceEmbeddingsDict.dict', 'rb') as fh:
    referenceEmbeddings = pickle.load(fh)

# ethnicity and disease membership vectors

In [10]:
ethnicityMembership = [case2ethnicity[c] if c in case2ethnicity else 'normal' for c in newEmbds]
ethnicityMembershipWN = [case2ethnicity[c] if c in case2ethnicity else 'normal' for c in wNormal]

diseaseMembership = [case2disease[c] if c in case2disease else 'normal' for c in newEmbds]
diseaseMembershipWN = [case2disease[c] if c in case2disease else 'normal' for c in wNormal]

# load spaces, masks based (for comparison with the original methodology)

In [11]:
noNormals = ['space_GPIBD_pairwise', 'space_GPIBD_correction1', 'space_GPIBD_both']
wNormals = ['space_GPIBD_pairwise_Normal', 'space_GPIBD_correction1_Normal', 'space_GPIBD_both_Normal']

def loadSpace(path):
    import pickle
    with open(path, 'rb') as fh:
        tmp = pickle.load(fh)
    return tmp

testDf_pairwise, testDf_split, testDf_both = list(map(loadSpace, noNormals))
testDf_pairwiseWN, testDf_splitWN, testDf_bothWN = list(map(loadSpace, wNormals))

# code for the new embeddings-based reference points

In [12]:
def makeReferencesProperLast(cases, membershipDictionary, references, k, mult = 1, scheme = 2):
    
    # makes new references the same way they were made for composite masks; 2 + 1 combinations of syndromic and ethnicity masks
    
    referenceMasksDict = dict()
    assignedK = int(k)
    
    for syn in set([*membershipDictionary.values()]):

    # select the cases that are of this syndrome
    # if there are less cases than is the assigned k value, take the num of cases as the k  
        tmpPool = [c for c in cases if membershipDictionary[c] == syn]
        if len(tmpPool) < k:
            k = len(tmpPool)
        else:
            k = assignedK
            
            
    #     as many times as there are cases create a reference mask (or 'mult' as many times)
        
        
        for i, _ in enumerate(tmpPool*mult):
            
            # if the pool is currently smaller than the k, replenish it
            if len(tmpPool) < k:
                tmpPool = [c for c in cases if membershipDictionary[c] == syn]
                
            tmpSample = [t for t in sample(tmpPool, k)]
            tmpAverage = np.average([cases[t] for t in tmpSample], axis=0)
            
            # remove the cases used in the current tmpAverage from the pool in order to give all cases equal participation
            for each in tmpSample:
                _ = tmpPool.pop(tmpPool.index(each))

    #         combine with all ethnicityReferenceMasks
            for mask,vec in references.items():
                
                tmpName = f'{i}_{syn}_{mask}'
                # the syndromic tmpAverage is calculated in twice in order to increase the syndromic effect;
                # hence the 2 + 1 scheme
                syndromicVec = [tmpAverage] * scheme
                referenceMasksDict[tmpName] = np.average(syndromicVec + [vec], axis=0)

    return referenceMasksDict

def makeReferencesProper3_1(cases, membershipDictionary, k, mult = 1):
    
    #### without ethnicity reference masks
    
    referenceMasksDict = dict()
    assignedK = int(k)
    
    for syn in set([*membershipDictionary.values()]):
    
        tmpPool = [c for c in cases if membershipDictionary[c] == syn]
        if len(tmpPool) < k:
            k = len(tmpPool)
        else:
            k = assignedK

        #     as many times as there are cases create a reference mask (or 'mult' as many times)
        
        for i, _ in enumerate(tmpPool*mult):

            # if the pool is currently smaller than the k, replenish it
            if len(tmpPool) < k:
                tmpPool = [c for c in cases if membershipDictionary[c] == syn]

            tmpSample = [t for t in sample(tmpPool, k)]
            tmpAverage = np.average([cases[t] for t in tmpSample], axis=0)

            # remove the cases used in the current tmpAverage from the pool in order to give all cases equal participation
            for each in tmpSample:
                _ = tmpPool.pop(tmpPool.index(each))

            tmpName = f'{i}_{syn}'

            referenceMasksDict[tmpName] = tmpAverage

    return referenceMasksDict

def transformDictToMatrix(d):
    
    d = {k:list(v[0]) for k,v in d.items()}
    df_d = pd.DataFrame.from_dict(d, orient = 'index')
    d = df_d.values
    
    return d

def pairwiseNP(cases):
    
    cases = transformDictToMatrix(cases)
    
    # snippet from: https://stackoverflow.com/questions/46700326/calculate-distances-between-one-point-in-matrix-from-all-other-points
    # study that
    
    res = np.linalg.norm(cases - cases[:,None], axis=-1)
    return res

def getAllIndices(x):

    composites = sorted(list(set([getComposite(n) for n in x.keys()])))
    indicesPerSet = list()
    
    for c in composites:
        tmpIndices = [i for i,k in enumerate(x) if c in k]
        indicesPerSet.append(tmpIndices)  
    
    print(len(indicesPerSet))
    
    return indicesPerSet

def getComposite(name):
   
    return '_'.join(name.split('_')[:3])

def getComposite2(name):
   
    return '_'.join(name.split('_')[3:])

def againstReferencesNP_old2(cases, references):
    
    # transform both to matrices
    
    X = transformDictToMatrix(cases)
    Y = transformDictToMatrix(references)
        
    # compute all the distances between these; 
    # the snippet for dists computing from: https://medium.com/dataholiks-distillery/l2-distance-matrix-vectorization-trick-26aa3247ac6c
    # study this
    
    # dists = np.sqrt(-2 * np.dot(X, Y.T) + np.sum(Y**2, axis=1) + np.sum(X**2, axis=1)[:, np.newaxis])
    dists = cdist(X,Y)
#     print(dists.shape)
    
    # get the indices of each set of gene composites and compute the minimum over that
    
    composites = sorted(list(set([getComposite(n) for n in references.keys()])))
    indicesPerSet = list()
    
    for c in composites:
#         print(c)
        gene = c.split('_')[1]
#         print(gene)
        tmpIndices = [i for i,k in enumerate(references) if gene in k]
        labelIndices = [k for i,k in enumerate(references) if gene in k]
        indicesPerSet.append(tmpIndices)
#         print(len(labelIndices))
#         print(labelIndices)
    
    finalDists = np.zeros((len(cases),len(indicesPerSet)))
    
    for i,t in enumerate(indicesPerSet):
        
        finalDists[:,i] = np.min(dists[:,t], 1)
    
    return finalDists

def againstReferencesNP_old3(cases, references):
    
    # transform both to matrices
    
    X = transformDictToMatrix(cases)
    Y = transformDictToMatrix(references)
        
    # compute all the distances between these; 
    # the snippet for dists computing from: https://medium.com/dataholiks-distillery/l2-distance-matrix-vectorization-trick-26aa3247ac6c
    # study this
    
    # dists = np.sqrt(-2 * np.dot(X, Y.T) + np.sum(Y**2, axis=1) + np.sum(X**2, axis=1)[:, np.newaxis])
    dists = cdist(X,Y)
#     print(dists.shape)
    
    # get the indices of each set of composites and compute the minimum over that
    
    return dists

# this bellow is simply for the comparison with the original spaces

# calculate the gene-based distance consistency measures for each of these spaces

In [11]:
spaces1 = [testDf_pairwise, testDf_split, testDf_both]

# gene lvl

In [12]:
for name, s in zip(noNormals, spaces1):
    tmp = PCA().fit_transform(s)
    print(f'space: {name}, {distConsistancy(tmp, set(diseaseMembership), diseaseMembership)}')

space: space_GPIBD_pairwise, ({'PGAP3': 0.56, 'PIGA': 0.4, 'PIGN': 0.2667, 'PIGV': 0.9583, 'GPAA1': 1.0, 'PIGU': 0.6, 'PIGT': 0.6154}, 0.6286285714285714)
space: space_GPIBD_correction1, ({'PGAP3': 0.68, 'PIGA': 0.16, 'PIGN': 0.1333, 'PIGV': 0.7083, 'GPAA1': 0.4, 'PIGU': 0.8, 'PIGT': 0.3846}, 0.46659999999999996)
space: space_GPIBD_both, ({'PGAP3': 0.6, 'PIGA': 0.16, 'PIGN': 0.1333, 'PIGV': 0.875, 'GPAA1': 0.8, 'PIGU': 0.6, 'PIGT': 0.6154}, 0.5405285714285714)


# ethn lvl

In [65]:
for name, s in zip(noNormals, spaces1):
    tmp = PCA().fit_transform(s)
    print(f'space: {name}, {distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))}')

space: space_GPIBD_pairwise, ({'AS': 0.3636, 'FIN': 1.0, 'EU+AFR': 1.0, 'ARAB': 1.0, 'EU': 0.3881, 'TURK': 0.4, 'MEX': 1.0, 'IND': 1.0, 'MID_EA': 0.5, 'NAF': 0.7059}, 0.73576)
space: space_GPIBD_correction1, ({'AS': 0.0909, 'FIN': 1.0, 'EU+AFR': 0.0, 'ARAB': 1.0, 'EU': 0.2239, 'TURK': 0.4, 'MEX': 0.0, 'IND': 1.0, 'MID_EA': 1.0, 'NAF': 0.6471}, 0.6702375)
space: space_GPIBD_both, ({'AS': 0.2727, 'FIN': 1.0, 'EU+AFR': 1.0, 'ARAB': 1.0, 'EU': 0.2537, 'TURK': 0.4, 'MEX': 0.0, 'IND': 1.0, 'MID_EA': 0.5, 'NAF': 0.7647}, 0.6879000000000001)


# check dsc for a simple all v all

In [19]:
X = transformDictToMatrixsformDictToMatrix(newEmbds)
Y = transformDictToMatrix(newEmbds)
dists_allVAll = cdist(X,Y)
tmp = PCA().fit_transform(dists_allVAll)
totalGene, avgGene = distConsistancy(tmp, set(diseaseMembership), diseaseMembership)
totalEthn, avgEthn = distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))
print(avgGene)
print(avgEthn)

0.6286285714285714
0.73576


# systematically calculate the distance consistency for all the ways you can create a space in: 

## no correction only; minimized over gene

In [72]:
bestAverageGene = 0
bestParamsGene = {'k':0, 'm':0}
bestAverageEthn = 1
bestParamsEthn = {'k':0, 'm':0}
bestDelta = 0
bestOverall = [0, 0]
bestOverallSpace = ''
bestParamsOverall = {'k':0, 'm':0}

trackGen_k_minimized = list()
trackGen_m_minimized = list()
trackGen_dsc_minimized = list()
trackEthn_dsc_minimized = list()

for k in tqdm_notebook(range(2, 6)):
    for m in tqdm_notebook(range(1, 5)):
#             print(f'k:{k}, m:{m}')
            
            references = makeReferencesProper3_1(newEmbds, case2disease, k, m)
            space1_noCorrection_minimized = againstReferencesNP_old2(newEmbds, references)
            
            # minimized = minimized over gene; might be worth looking into minimizing over correction masks again
            
            spaces = [space1_noCorrection_minimized]
            
            spaceNames = ['embeddingsBased_noCorrection_minimized']
            
            for name, space in zip(spaceNames, spaces):
                tmp = PCA().fit_transform(space)
                totalGene, avgGene = distConsistancy(tmp, set(diseaseMembership), diseaseMembership)
                totalEthn, avgEthn = distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))
                
                # save the scores in respect to the two parameters and the parameters
                trackGen_k_minimized.append(k)
                trackGen_m_minimized.append(m)
                trackGen_dsc_minimized.append(avgGene)
                trackEthn_dsc_minimized.append(avgEthn)
                
#                 print(f'Gene__space: {name}, {avgGene}')
#                 print(f'Ethn__space: {name}, {avgEthn}')
                if avgGene > bestAverageGene:
                    bestAverageGene = avgGene
                    bestParamsGene['k'], bestParamsGene['m'] = k, m
                if avgEthn < bestAverageEthn:
                    bestAverageEthn = avgEthn
                    bestParamsEthn['k'], bestParamsEthn['m'] = k, m
                if (avgGene - avgEthn) > bestDelta:
                    bestDelta = avgGene - avgEthn
                    bestOverall[0], bestOverall[1] = avgGene, avgEthn
                    bestParamsOverall['k'], bestParamsOverall['m'] = k, m
                    bestOverallSpace = name

In [74]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.9144857142857142
bestParamsGene {'k': 2, 'm': 3}
bestAverageEthn 0.501511111111111
bestParamsEthn {'k': 5, 'm': 3}
bestDelta 0.22186964285714295
bestOverall [0.8444571428571429, 0.6225875]
bestParamsOverall {'k': 3, 'm': 2}
bestOverallSpace embeddingsBased_noCorrection_minimized


# surface plot the dsc in respect to these two parameters

In [178]:
def plotSurface(kArray, mArray, genDscArray, ethnDscArray, titl):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import axes3d, Axes3D
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    fig = plt.figure(figsize=(15,10))

    ax = Axes3D(fig)
    #gene lvl
    x, y, z = np.array(kArray), np.array(mArray), np.array(genDscArray)
    ax1 = ax.plot_trisurf(x, y, z, linewidth=0, antialiased=False, cmap='plasma', edgecolor='none', alpha = 0.75,
                          vmin=min(z), vmax=max(z))

    #ethn lvl
    x, y, z = np.array(kArray), np.array(mArray), np.array(ethnDscArray)
    ax2 = ax.plot_trisurf(x, y, z, linewidth=0, antialiased=False, cmap='viridis_r', edgecolor='none', alpha = 0.75,
                        vmin=min(z), vmax=max(z))

    cbar1 = fig.colorbar(ax1, orientation='vertical', shrink=0.8)
    cbar1.set_label('Gene', fontsize = 15)

    cbar2 = fig.colorbar(ax2, orientation='vertical', shrink=0.8)
    cbar2.set_label('Ethnicity', fontsize = 15)

    ax.set_xlabel('k')
    ax.set_ylabel('m')
    ax.set_zlabel('dsc');
    ax.set_title(titl, fontsize = 20)

    # fig.colorbar(im, orientation='vertical', shrink=0.8)
    plt.show()

In [179]:
plotSurface(trackGen_k_minimized, trackGen_m_minimized, trackGen_dsc_minimized, trackEthn_dsc_minimized, titl = 'Distance consistency;\nno correction, minimized over gene')

# systematically calculate the distance consistency for all the ways you can create a space in:

## no correction only; allVAll

In [161]:
bestAverageGene = 0
bestParamsGene = {'k':0, 'm':0}
bestAverageEthn = 1
bestParamsEthn = {'k':0, 'm':0}
bestDelta = 0
bestOverall = [0, 0]
bestOverallSpace = ''
bestParamsOverall = {'k':0, 'm':0}

# all v all, simple cdist(X,Y)-based distance computation; to show the power in minimizing over gene

trackGen_k_allV = list()
trackGen_m_allV = list()
trackGen_dsc_allV = list()
trackEthn_dsc_allV = list()

for k in tqdm_notebook(range(2, 6)):
    for m in tqdm_notebook(range(1, 5)):
#             print(f'k:{k}, m:{m}')
            
            references = makeReferencesProper3_1(newEmbds, case2disease, k, m)
            space1_noCorrection_allVall = againstReferencesNP_old3(newEmbds, references)
            
            # minimized = minimized over gene; might be worth looking into minimizing over correction masks again
            
            spaces = [space1_noCorrection_allVall]
            
            spaceNames = ['embeddingsBased_noCorrection_AllvAll']
            
            for name, space in zip(spaceNames, spaces):
                tmp = PCA().fit_transform(space)
                totalGene, avgGene = distConsistancy(tmp, set(diseaseMembership), diseaseMembership)
                totalEthn, avgEthn = distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))
                
                # save the scores in respect to the two parameters
                trackGen_k_allV.append(k)
                trackGen_m_allV.append(m)
                trackGen_dsc_allV.append(avgGene)
                trackEthn_dsc_allV.append(avgEthn)
                
#                 print(f'Gene__space: {name}, {avgGene}')
#                 print(f'Ethn__space: {name}, {avgEthn}')
                if avgGene > bestAverageGene:
                    bestAverageGene = avgGene
                    bestParamsGene['k'], bestParamsGene['m'] = k, m
                if avgEthn < bestAverageEthn:
                    bestAverageEthn = avgEthn
                    bestParamsEthn['k'], bestParamsEthn['m'] = k, m
                if (avgGene - avgEthn) > bestDelta:
                    bestDelta = avgGene - avgEthn
                    bestOverall[0], bestOverall[1] = avgGene, avgEthn
                    bestParamsOverall['k'], bestParamsOverall['m'] = k, m
                    bestOverallSpace = name

In [163]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.5809714285714287
bestParamsGene {'k': 2, 'm': 2}
bestAverageEthn 0.5249
bestParamsEthn {'k': 5, 'm': 4}
bestDelta 0.0015873015873016927
bestOverall [0.5464428571428572, 0.5448555555555555]
bestParamsOverall {'k': 3, 'm': 2}
bestOverallSpace embeddingsBased_noCorrection_AllvAll


# plot

In [183]:
plotSurface(trackGen_k_allV, trackGen_m_allV, trackGen_dsc_allV, trackEthn_dsc_allV, 'Distance consistency;\nno correction, all V all')

# check the stability of the space

In [20]:
from scipy.stats import describe

In [23]:
genDsc = list()
ethnDsc = list()

for _ in tqdm_notebook(range(50)):
    references = makeReferencesProperLast(newEmbds, case2disease, referenceEmbeddings, bestParamsOverall['k'],
                                          bestParamsOverall['m'], bestParamsOverall['s'])
    space2_withCorrection_minimized = againstReferencesNP_old2(newEmbds, references)
    tmp = PCA().fit_transform(space2_withCorrection_minimized)
    totalGene, avgGene = distConsistancy(tmp, set(diseaseMembership), diseaseMembership)
    totalEthn, avgEthn = distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))
    genDsc.append(avgGene)
    ethnDsc.append(avgEthn)

In [26]:
print(describe(genDsc))
print(describe(ethnDsc))
print(describe([i-j for i,j in zip(genDsc, ethnDsc)]))

DescribeResult(nobs=50, minmax=(0.7851714285714284, 0.8349285714285715), mean=0.8087505714285713, variance=0.00010318542448979658, skewness=0.24915818869680662, kurtosis=0.018789491527671842)
DescribeResult(nobs=50, minmax=(0.53118, 0.6945888888888888), mean=0.6224558444444445, variance=0.0010060994357173091, skewness=-0.2424906716400856, kurtosis=0.7036187804263583)
DescribeResult(nobs=50, minmax=(0.1102063492063492, 0.28279142857142847), mean=0.1862947269841269, variance=0.0009939209185291006, skewness=0.2235723120348155, kurtosis=0.9552423449948777)


# with correction; minimized over gene

In [195]:
bestAverageGene = 0
bestParamsGene = {'k':0, 'm':0}
bestAverageEthn = 1
bestParamsEthn = {'k':0, 'm':0}
bestDelta = 0
bestOverall = [0, 0]
bestOverallSpace = ''
bestParamsOverall = {'k':0, 'm':0}

trackGen_k_minimized = list()
trackGen_m_minimized = list()
trackGen_dsc_minimized = list()
trackEthn_dsc_minimized = list()

##########
s = 5 #### Fixed s so we can plot the dsc in respect to k and m
##########

for k in tqdm_notebook(range(2, 6)):
    for m in tqdm_notebook(range(1, 5)):
            
            references = makeReferencesProperLast(newEmbds, case2disease, referenceEmbeddings, k, m, s)
            space2_withCorrection_minimized = againstReferencesNP_old2(newEmbds, references)
            
            # minimized = minimized over gene; might be worth looking into minimizing over correction masks again
            
            spaces = [space2_withCorrection_minimized]
            
            spaceNames = ['embeddingsBased_withCorrection_minimized']
            
            for name, space in zip(spaceNames, spaces):
                tmp = PCA().fit_transform(space)
                totalGene, avgGene = distConsistancy(tmp, set(diseaseMembership), diseaseMembership)
                totalEthn, avgEthn = distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))
                
                # save the scores in respect to the parameters
                trackGen_k_minimized.append(k)
                trackGen_m_minimized.append(m)
                trackGen_dsc_minimized.append(avgGene)
                trackEthn_dsc_minimized.append(avgEthn)
                
#                 print(f'Gene__space: {name}, {avgGene}')
#                 print(f'Ethn__space: {name}, {avgEthn}')
                if avgGene > bestAverageGene:
                    bestAverageGene = avgGene
                    bestParamsGene['k'], bestParamsGene['m'] = k, m
                if avgEthn < bestAverageEthn:
                    bestAverageEthn = avgEthn
                    bestParamsEthn['k'], bestParamsEthn['m'] = k, m
                if (avgGene - avgEthn) > bestDelta:
                    bestDelta = avgGene - avgEthn
                    bestOverall[0], bestOverall[1] = avgGene, avgEthn
                    bestParamsOverall['k'], bestParamsOverall['m'] = k, m
                    bestOverallSpace = name

# s = 2

In [185]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.8387428571428571
bestParamsGene {'k': 2, 'm': 4}
bestAverageEthn 0.56965
bestParamsEthn {'k': 5, 'm': 1}
bestDelta 0.24692000000000003
bestOverall [0.8328, 0.58588]
bestParamsOverall {'k': 2, 'm': 1}
bestOverallSpace embeddingsBased_withCorrection_minimized


# plot

In [186]:
plotSurface(trackGen_k_minimized, trackGen_m_minimized, trackGen_dsc_minimized, trackEthn_dsc_minimized, 'Distance consistency\nWith correction, minimized, s = 2')

# s = 3

In [189]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.8615999999999999
bestParamsGene {'k': 2, 'm': 3}
bestAverageEthn 0.5315
bestParamsEthn {'k': 5, 'm': 1}
bestDelta 0.22566142857142846
bestOverall [0.8520714285714285, 0.62641]
bestParamsOverall {'k': 2, 'm': 2}
bestOverallSpace embeddingsBased_withCorrection_minimized


# plot

In [190]:
plotSurface(trackGen_k_minimized, trackGen_m_minimized, trackGen_dsc_minimized, trackEthn_dsc_minimized, 'Distance consistency\nWith correction, minimized, s = 3')

# s = 4

In [193]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.8916285714285713
bestParamsGene {'k': 2, 'm': 2}
bestAverageEthn 0.52712
bestParamsEthn {'k': 5, 'm': 2}
bestDelta 0.2677128571428571
bestOverall [0.8806428571428571, 0.61293]
bestParamsOverall {'k': 2, 'm': 3}
bestOverallSpace embeddingsBased_withCorrection_minimized


# plot

In [194]:
plotSurface(trackGen_k_minimized, trackGen_m_minimized, trackGen_dsc_minimized, trackEthn_dsc_minimized, 'Distance consistency\nWith correction, minimized, s = 4')

# s = 5

In [196]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.8916285714285713
bestParamsGene {'k': 2, 'm': 2}
bestAverageEthn 0.5486555555555556
bestParamsEthn {'k': 5, 'm': 2}
bestDelta 0.22130142857142854
bestOverall [0.7739714285714285, 0.55267]
bestParamsOverall {'k': 4, 'm': 2}
bestOverallSpace embeddingsBased_withCorrection_minimized


# plot

In [197]:
plotSurface(trackGen_k_minimized, trackGen_m_minimized, trackGen_dsc_minimized, trackEthn_dsc_minimized, 'Distance consistency\nWith correction, minimized, s = 5')

# with correction; all v all

In [202]:
bestAverageGene = 0
bestParamsGene = {'k':0, 'm':0}
bestAverageEthn = 1
bestParamsEthn = {'k':0, 'm':0}
bestDelta = 0
bestOverall = [0, 0]
bestOverallSpace = ''
bestParamsOverall = {'k':0, 'm':0}

# all v all, simple cdist(X,Y)-based distance computation; to show the power in minimizing over gene

trackGen_k_allV = list()
trackGen_m_allV = list()
trackGen_dsc_allV = list()
trackEthn_dsc_allV = list()

##########
s = 4  ### Fixed so we can plot the dsc in respect to k and m
##########

for k in tqdm_notebook(range(2, 6)):
    for m in tqdm_notebook(range(1, 5)):
#             print(f'k:{k}, m:{m}, s:{s}')
            
            references = makeReferencesProperLast(newEmbds, case2disease, referenceEmbeddings, k, m, s)
            space2_withCorrection_allVall = againstReferencesNP_old3(newEmbds, references)
            
            # minimized = minimized over gene; might be worth looking into minimizing over correction masks again
            
            spaces = [space2_withCorrection_allVall]
            
            spaceNames = ['embeddingsBased_withCorrection_AllvAll']
            
            for name, space in zip(spaceNames, spaces):
                tmp = PCA().fit_transform(space)
                totalGene, avgGene = distConsistancy(tmp, set(diseaseMembership), diseaseMembership)
                totalEthn, avgEthn = distConsistancy(tmp, set(case2ethnicity.values()), list(case2ethnicity.values()))
                
                # save the scores in respect to the parameters
                # save the scores in respect to the two parameters
                trackGen_k_allV.append(k)
                trackGen_m_allV.append(m)
                trackGen_dsc_allV.append(avgGene)
                trackEthn_dsc_allV.append(avgEthn)
                
#                 print(f'Gene__space: {name}, {avgGene}')
#                 print(f'Ethn__space: {name}, {avgEthn}')
                if avgGene > bestAverageGene:
                    bestAverageGene = avgGene
                    bestParamsGene['k'], bestParamsGene['m'] = k, m
                if avgEthn < bestAverageEthn:
                    bestAverageEthn = avgEthn
                    bestParamsEthn['k'], bestParamsEthn['m'] = k, m
                if (avgGene - avgEthn) > bestDelta:
                    bestDelta = avgGene - avgEthn
                    bestOverall[0], bestOverall[1] = avgGene, avgEthn
                    bestParamsOverall['k'], bestParamsOverall['m'] = k, m
                    bestOverallSpace = name

# s = 2

In [200]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.4945428571428572
bestParamsGene {'k': 2, 'm': 3}
bestAverageEthn 0.5768899999999999
bestParamsEthn {'k': 5, 'm': 1}
bestDelta 0
bestOverall [0, 0]
bestParamsOverall {'k': 0, 'm': 0}
bestOverallSpace 


# plot

In [201]:
plotSurface(trackGen_k_allV, trackGen_m_allV, trackGen_dsc_allV, trackEthn_dsc_allV, 'Distance consistency;\nno correction, all V all, s = 2')

# s = 4

In [203]:
dscVals = ['bestAverageGene',
'bestParamsGene',
'bestAverageEthn',
'bestParamsEthn',
'bestDelta',
'bestOverall',
'bestParamsOverall',
'bestOverallSpace']
for v in dscVals:
    print(f'{v} {eval(v)}')

bestAverageGene 0.5367285714285714
bestParamsGene {'k': 2, 'm': 3}
bestAverageEthn 0.51653
bestParamsEthn {'k': 5, 'm': 4}
bestDelta 0
bestOverall [0, 0]
bestParamsOverall {'k': 0, 'm': 0}
bestOverallSpace 


# plot

In [204]:
plotSurface(trackGen_k_allV, trackGen_m_allV, trackGen_dsc_allV, trackEthn_dsc_allV, 'Distance consistency;\nno correction, all V all, s = 4')

# leave one out; different classifiers

In [36]:
def sortUnique(n):
    n = np.array(n)
    _, idx = np.unique(n, return_index=True)
    n = n[np.sort(idx)]
    return list(n)

In [14]:
def makeClassifiers():
    
    clsNames = ['linearSvm1', 'linear2', 'linear3', 'kernelSvm3', 'kernelSvm4']
    cls = [
    SVC(kernel="linear", C= 1, cache_size=300),
    SVC(kernel='linear', C = 10, cache_size=300),
    SVC(kernel='linear', C = 15, cache_size=300),
    SVC(kernel='rbf', C = 10, gamma=0.001, cache_size=300),
    SVC(kernel='rbf', C = 1000, gamma=0.001, cache_size=300)]
    
    return clsNames, cls

In [15]:
from sklearn.model_selection import GridSearchCV

In [19]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

for k in range(4, 5):
    for m in range(1, 5):
        
        kf = KFold(n_splits=len(newEmbds))
        kf.get_n_splits(newEmbds)

        totalScore = list()
        bar = tqdm_notebook(kf.split(newEmbds), total = len(newEmbds))

        clsNames, _ = makeClassifiers()
        scores1 = {k:list() for k in clsNames}
#         scores = list()

        for train_index, test_index in bar:

            trainCases = {k:v for i, (k,v) in enumerate(newEmbds.items()) if i not in test_index}
            trainLabels = list({k:v for k,v in case2disease.items() if k in trainCases}.values())

            testKey = list(newEmbds.keys())[test_index[0]]
            testCases = {testKey: newEmbds[testKey]}
            testLabels = [(case2disease[testKey])]

            ## parameters selected because of the best space above; this is best overall
#             references_wCorrection = makeReferencesProperLast(trainCases, case2disease, referenceEmbeddings, k, m, 4)
#             distsTrain1 = againstReferencesNP_old3(trainCases, references_wCorrection)
#             distsTest1 = againstReferencesNP_old3(testCases, references_wCorrection)
        
            references_NoCorrection = makeReferencesProper3_1(trainCases, case2disease, k, m)
            distsTrain1 = againstReferencesNP_old3(trainCases, references_NoCorrection)      
            distsTest1 = againstReferencesNP_old3(testCases, references_NoCorrection)

#             classfiers:
            clsNames, cls = makeClassifiers()
            
            for name, cl in zip(clsNames, cls):
                cl.fit(distsTrain1, trainLabels)
                scores1[name].append(cl.score(distsTest1, testLabels))
            
        print(k, m)
        for key, val in scores1.items():
            print(key, np.mean(val))
        print()
        print()

4 1
linearSvm1 0.5803571428571429
linear2 0.5625
linear3 0.5625
kernelSvm3 0.16964285714285715
kernelSvm4 0.5714285714285714




4 2
linearSvm1 0.5892857142857143
linear2 0.5357142857142857
linear3 0.5357142857142857
kernelSvm3 0.4642857142857143
kernelSvm4 0.5714285714285714




KeyboardInterrupt: 